TODO:
1. Parse EE metadata

Metadata collection workflow:

1. Collect metadata from logs. This includes date the image was pulled; glims_id; crs; utm_zone; landsat image collection;

2. Collect metadata from metadata files for each landsat

3. Collect metadata from the image files. This includes the date the image was taken; the file size; 

4. Collect metadata from the images. This includes the image shape.

5. Join on filename

{'type': 'Image',
 'bands': [
     {'id': 'B1',
      'data_type': {'type': 'PixelType', 'precision': 'float'}
      'dimensions': [8441, 7881],
      'crs': 'EPSG:32632',
      'crs_transform': [30, 0, 297885, 0, -30, 6934215]
     },
     {'id': 'B2',
      'data_type': {'type': 'PixelType', 'precision': 'float'},
      'dimensions': [8441, 7881],
      'crs': 'EPSG:32632',
      'crs_transform': [30, 0, 297885, 0, -30, 6934215]
     },
     {'id': 'B3',
      'data_type': {'type': 'PixelType', 'precision': 'float'},
      'dimensions': [8441, 7881],
      'crs': 'EPSG:32632',
      'crs_transform': [30, 0, 297885, 0, -30, 6934215]
     },
     {'id': 'B4',
      'data_type': {'type': 'PixelType', 'precision': 'float'},
      'dimensions': [8441, 7881],
      'crs': 'EPSG:32632',
      'crs_transform': [30, 0, 297885, 0, -30, 6934215]
     },
     {'id': 'B5',
      'data_type': {'type': 'PixelType', 'precision': 'float'},
      'dimensions': [8441, 7881],
      'crs': 'EPSG:32632',
      'crs_transform': [30, 0, 297885, 0, -30, 6934215]
     }, 
     {'id': 'B6',
      'data_type': {'type': 'PixelType', 'precision': 'float'},
      'dimensions': [8441, 7881],
      'crs': 'EPSG:32632',
      'crs_transform': [30, 0, 297885, 0, -30, 6934215]
     },
     {'id': 'B7',
      'data_type': {'type': 'PixelType', 'precision': 'float'},
      'dimensions': [8441, 7881],
      'crs': 'EPSG:32632',
      'crs_transform': [30, 0, 297885, 0, -30, 6934215]
     }, {'id': 'BQA',
         'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
         'dimensions': [8441, 7881],
         'crs': 'EPSG:32632',
         'crs_transform': [30, 0, 297885, 0, -30, 6934215]
        }
 ],
 'version': 1637547863240067,
 'id': 'LANDSAT/LT05/C01/T1_TOA/LT05_200017_20060808',
 'properties': {#'cloud': 71.90979999905855,
                'RADIANCE_MULT_BAND_5': 0.12035000324249268,
                'RADIANCE_MULT_BAND_6': 0.05537499859929085,
                #'IMAGE_QUALITY': 9,
                'RADIANCE_MULT_BAND_3': 1.0440000295639038,
                'RADIANCE_MULT_BAND_4': 0.8760200142860413,
                'RADIANCE_MULT_BAND_1': 0.7658299803733826,
                'RADIANCE_MULT_BAND_2': 1.448199987411499,
                'K2_CONSTANT_BAND_6': 1260.56005859375,
                'CORRECTION_GAIN_BAND_2': 'CPF',
                'CORRECTION_GAIN_BAND_1': 'CPF',
                'system:footprint': {'type': 'LinearRing',
                                     'coordinates': [[8.75944773267767, 60.42588278654468],
                                                     [8.78290786252994, 60.4220613280782],
                                                     [8.803640478497163, 60.427365404461185],
                                                     [9.010315528367336, 60.69830918687514],
                                                     [9.725522968205079, 61.610700364371446],
                                                     [9.96733652254739, 61.91065091536024],
                                                     [9.973336904189752, 61.91873371071608],
                                                     [9.974303257410686, 61.94909094021114],
                                                     [9.935413704121657, 61.959954475179025],
                                                     [9.87867690153891, 61.96954169855863], 
                                                     [9.777569658103324, 61.986259157961335],
                                                     [8.62685296649254, 62.16999082828351],
                                                     [6.267470163020758, 62.510551121272854],
                                                     [6.265117539693487, 62.510505024024326],
                                                     [5.752410602905267, 61.761351318119715],
                                                     [5.262630047720833, 61.00894294544531],
                                                     [5.260758849404971, 61.002948628629916],
                                                     [5.267110974159161, 60.94888832276869],
                                                     [5.342406950071589, 60.938577055995644],
                                                     [7.6122223377085065, 60.6096061920694],
                                                     [7.992764910838719, 60.5499790898727],
                                                     [8.75944773267767, 60.42588278654468]]
                                    },
                'REFLECTIVE_SAMPLES': 8441,
                'CORRECTION_GAIN_BAND_7': 'CPF',
                'CORRECTION_GAIN_BAND_6': 'INTERNAL_CALIBRATION',
                'CORRECTION_GAIN_BAND_5': 'CPF',
                'SUN_AZIMUTH': 160.42684936523438,
                'CORRECTION_GAIN_BAND_4': 'CPF',
                'CORRECTION_GAIN_BAND_3': 'CPF',
                'CPF_NAME': 'LT05CPF_20060701_20060815_01.03',
                'DATE_ACQUIRED': '2006-08-08',
                'ELLIPSOID': 'WGS84',
                'SENSOR_MODE': 'BUMPER',
                'STATION_ID': 'KIS',
                'RESAMPLING_OPTION': 'CUBIC_CONVOLUTION',
                'ORIENTATION': 'NORTH_UP',
                'WRS_ROW': 17,
                'RADIANCE_MULT_BAND_7': 0.06555099785327911,
                #'CLOUD_COVER': 52,
                'GEOMETRIC_RMSE_VERIFY': 0.5109999775886536,
                'COLLECTION_CATEGORY': 'T1',
                'GRID_CELL_SIZE_REFLECTIVE': 30,
                'DATA_CATEGORY': 'NOMINAL',
                'K1_CONSTANT_BAND_6': 607.760009765625,
                'CLOUD_COVER_LAND': 52,
                'GEOMETRIC_RMSE_MODEL': 8.10200023651123,
                'COLLECTION_NUMBER': 1,
                'LANDSAT_SCENE_ID': 'LT52000172006220KIS01',
                'WRS_PATH': 200,
                'MAP_PROJECTION_L0RA': 'NA',
                'GEOMETRIC_RMSE_MODEL_Y': 7.4079999923706055,
                'REFLECTIVE_LINES': 7881,
                'GEOMETRIC_RMSE_MODEL_X': 3.2799999713897705,
                'system:asset_size': 261899008,
                'system:index': 'LT05_200017_20060808',
                'GEOMETRIC_RMSE_VERIFY_QUAD_LL': 0.4429999887943268,
                'GEOMETRIC_RMSE_VERIFY_QUAD_LR': 0.5860000252723694,
                'REFLECTANCE_ADD_BAND_1': -0.0037980000488460064,
                'REFLECTANCE_ADD_BAND_2': -0.007873999886214733,
                'DATUM': 'WGS84',
                'REFLECTANCE_ADD_BAND_3': -0.004798999987542629,
                'REFLECTANCE_ADD_BAND_4': -0.007460999768227339,
                'REFLECTANCE_ADD_BAND_5': -0.007557000033557415,
                'REFLECTANCE_ADD_BAND_7': -0.008465999737381935,
                'GROUND_CONTROL_POINTS_VERSION': 4,
                'DATA_TYPE': 'L1TP',
                'GEOMETRIC_RMSE_VERIFY_QUAD_UL': 0.48100000619888306,
                'CORRECTION_BIAS_BAND_2': 'CPF',
                'CORRECTION_BIAS_BAND_1': 'CPF',
                'GEOMETRIC_RMSE_VERIFY_QUAD_UR': 0.4729999899864197,
                'CORRECTION_BIAS_BAND_4': 'CPF',
                'CORRECTION_BIAS_BAND_3': 'CPF',
                'DATA_TYPE_L0RP': 'TMR_L0RP',
                'UTM_ZONE': 32,
                'LANDSAT_PRODUCT_ID': 'LT05_L1TP_200017_20060808_20161119_01_T1',
                'RADIANCE_ADD_BAND_4': -2.3860199451446533,
                'REFLECTANCE_MULT_BAND_7': 0.0025746000465005636,
                'system:time_start': 1155033406830,
                'RADIANCE_ADD_BAND_5': -0.49035000801086426,
                'RADIANCE_ADD_BAND_6': 1.1824300289154053,
                'PROCESSING_SOFTWARE_VERSION': 'LPGS_12.8.2',
                'RADIANCE_ADD_BAND_7': -0.2155500054359436,
                'EPHEMERIS_TYPE': 'DEFINITIVE',
                'RADIANCE_ADD_BAND_1': -2.285830020904541,
                'RADIANCE_ADD_BAND_2': -4.288189888000488,
                'RADIANCE_ADD_BAND_3': -2.213979959487915,
                'REFLECTANCE_MULT_BAND_1': 0.0012725000269711018,
                'CORRECTION_BIAS_BAND_6': 'CPF',
                'REFLECTANCE_MULT_BAND_3': 0.0022630998864769936, 
                'CORRECTION_BIAS_BAND_5': 'CPF',
                'REFLECTANCE_MULT_BAND_2': 0.002659300109371543,
                'REFLECTANCE_MULT_BAND_5': 0.0018547000363469124,
                'CORRECTION_BIAS_BAND_7': 'CPF',
                'REFLECTANCE_MULT_BAND_4': 0.0027391999028623104,
                'THERMAL_LINES': 7881,
                'GRID_CELL_SIZE_THERMAL': 30,
                'REQUEST_ID': '0501611191195_06050',
                'EARTH_SUN_DISTANCE': 1.0139789581298828,
                'FILE_DATE': 1479590848000,
                'SCENE_CENTER_TIME': '10:36:46.8280000Z',
                'SUN_ELEVATION': 43.447120666503906,
                'SATURATION_BAND_1': 'Y', 
                'SATURATION_BAND_2': 'Y',
                'SATURATION_BAND_3': 'Y',
                'SATURATION_BAND_4': 'Y',
                'SATURATION_BAND_5': 'Y',
                'MAP_PROJECTION': 'UTM',
                'SATURATION_BAND_6': 'N',
                'SENSOR_ID': 'TM',
                'SATURATION_BAND_7': 'Y',
                'GROUND_CONTROL_POINTS_MODEL': 38,
                'SPACECRAFT_ID': 'LANDSAT_5',
                'ELEVATION_SOURCE': 'GLS2000',
                'THERMAL_SAMPLES': 8441,
                'GROUND_CONTROL_POINTS_VERIFY': 555
               }
}

In [21]:
import pandas as pd
import os
import rasterio
import sys
import json
import read

In [23]:
output_dir = "/Users/mattw/Desktop/projects/GlacierView/src/earth_engine/data/processed_metadata/localized_time_series_for_segmentation_training_large"

#don't have this for the large traning data
#logs_path = "/Users/mattw/Desktop/projects/GlacierView/src/earth_engine/data/ee_landing_zone/full_time_series/logs 

glacier_view_dir = os.path.join(os.path.expanduser("~"),"Desktop","projects","GlacierView")
glaciers_dir = os.path.join(glacier_view_dir,"src","earth_engine","data","ee_landing_zone/localized_time_series_for_segmentation_training_large/landsat")

### Step 1: Collect and clean logs

In [6]:
batch_1_metadata_path =  os.path.join(glacier_view_dir,"data","ee_data","training_batches","batch_1","training_log.log")
batch_2_metadata_path =  os.path.join(glacier_view_dir,"data","ee_data","training_batches","batch_2","training_log.log")

In [7]:
GLIMS_ID_STR_INDEX = 1
CRS_STR_INDEX = 2
UTM_ZONE_STR_INDEX = 1

In [26]:
batch_1_df = pd.read_csv(batch_1_metadata_path, sep = ",", header = None)
batch_2_df = pd.read_csv(batch_2_metadata_path, sep = ",", header = None)
log_df = pd.concat((batch_1_df, batch_2_df), axis = 0)
log_df.columns = ['ee_pull_date', 'glims_id','crs','utm_zone','landsat_satellite']
log_df.glims_id = log_df.glims_id.str.split(": ", expand = True)[GLIMS_ID_STR_INDEX] 
log_df.crs = log_df.crs.str.split(":", expand = True)[CRS_STR_INDEX]
log_df.utm_zone = log_df.utm_zone.str.split(": ", expand = True)[UTM_ZONE_STR_INDEX]

In [27]:
log_df.to_csv(os.path.join(output_dir,"processed_logs.csv"), index = False)

### Step 2: Collect image file sizes (filename, file size in bytes)

In [17]:
glims_ids = os.listdir(glaciers_dir) 
glaciers_sizes = []
for glims_id in glims_ids:
    glacier_dir = os.path.join(glaciers_dir,glims_id)
    try:
        folder = os.listdir(glacier_dir)
    except:
        continue
    for glacier in folder:
        if glacier == "meta_data":
            continue
        base_name = glacier.split(".")[0]
        glacier_path = os.path.join(glacier_dir,glacier)
        size_in_bytes = os.path.getsize(glacier_path)
        glaciers_sizes.append((base_name,size_in_bytes)) 

In [18]:
size_df = pd.DataFrame(glaciers_sizes, columns = ["glacier_pk", "file size in bytes"])

In [19]:
size_df.to_csv(os.path.join(output_dir,"file_sizes.csv"), index = False)

### Step 3: Collect image shapes, extents and other metadata

In [25]:
image_properties = []
for glims_id in glims_ids:
    glacier_dir = os.path.join(glaciers_dir,glims_id)
    glacier_file_names = os.listdir(glacier_dir)
    glacier_file_names = [glac for glac in glacier_file_names if len(glac) > 10] #ignores .DS store
    
    rasters = read.get_rasters(glacier_file_names)
    extents = read.get_extents(glaciers_dir, glacier_file_names)
    metainfo = read.get_metainfo(glaciers_dir, glacier_file_names)
    
    for idx,raster in enumerate(rasters):
        base_name = glacier_file_names[idx].split(".")[0]
        image_properties.append((base_name, extents[idx], metainfo[idx]))

TypeError: expected str, bytes or os.PathLike object, not list

In [54]:
df_image_props = pd.DataFrame(image_properties, columns = ["glacier_pk", "extent", "metainfo"])

In [55]:
raster_metadata_df = pd.DataFrame(columns = ['driver', 'dtype', 'nodata', 'width', 'height', 'count', 'crs'])
extent_df = pd.DataFrame(columns = ['left','bottom','right','top'])
for idx,row in df_image_props.iterrows():
    metainfo = row['metainfo'].copy()
    extent = list(row['extent'])
    extent = dict(zip(["left","bottom","right","top"],extent))
    del metainfo['transform']
    metainfo['crs'] = str(metainfo['crs'])
    raster_metadata_df = raster_metadata_df.append(metainfo, ignore_index = True)
    extent_df = extent_df.append(extent, ignore_index = True)

In [56]:
df_image_props = pd.concat((df_image_props, raster_metadata_df, extent_df), axis = 1)

In [57]:
df_image_props = df_image_props.drop(["metainfo","extent"], axis = 1)

In [58]:
df_image_props

,glacier_pk,driver,dtype,nodata,width,height,count,crs,left,bottom,right,top
0,G005644E61747N_2006-08-22_L5_T1_TOA,GTiff,float32,None,98,87,8,EPSG:32632,321465.0,6849765.0,324405.0,6852375.0
1,G005644E61747N_2006-10-10_L7_T1_TOA,GTiff,float32,None,97,86,10,EPSG:32631,638205.0,6848055.0,641115.0,6850635.0
2,G005644E61747N_2006-09-15_L7_T1_TOA,GTiff,float32,None,98,87,10,EPSG:32632,321465.0,6849765.0,324405.0,6852375.0
3,G005644E61747N_2006-09-23_L5_T1_TOA,GTiff,float32,None,98,87,8,EPSG:32632,321465.0,6849765.0,324405.0,6852375.0
4,G005644E61747N_2006-08-23_L7_T1_TOA,GTiff,float32,None,97,86,10,EPSG:32631,638205.0,6848055.0,641115.0,6850635.0
...,...,...,...,...,...,...,...,...,...,...,...,...
36589,G324056E54773S_2016-04-07_L8_T1_TOA,GTiff,float32,None,269,217,12,EPSG:32624,692685.0,-6076845.0,700755.0,-6070335.0
36590,G324056E54773S_2016-03-06_L8_T1_TOA,GTiff,float32,None,269,217,12,EPSG:32624,692685.0,-6076845.0,700755.0,-6070335.0
36591,G324056E54773S_2016-03-22_L8_T1_TOA,GTiff,float32,None,269,217,12,EPSG:32624,692685.0,-6076845.0,700755.0,-6070335.0
36592,G324056E54773S_2016-01-02_L8_T1_TOA,GTiff,float32,None,269,217,12,EPSG:32624,692685.0,-6076845.0,700755.0,-6070335.0


In [59]:
df_image_props.to_csv(os.path.join(training_metadata_write_dir,"image_properties.csv"), index = False)

### Step 4: Collect JSON metadata from EE

In [26]:
import json
import os

In [27]:
glaciers = os.listdir(glaciers_dir)
ee_meta_data_df = pd.DataFrame({"glacier_pk":[], 
                               "cloud":[], 
                               "cloud_cover": [],
                               "image_quality": [],
                               "image_quality_tirs": [],
                               "iamge_quality_oli": [],
                               "cloud_cover_land": [],
                               "system_index": [],
                               "utm_zone": []
                              })
for glacier in glaciers:
    if glacier == ".DS_Store": continue
    glacier_dir = os.path.join(glaciers_dir, glacier)
    if "meta_data" not in os.listdir(glacier_dir): continue
    meta_data_dir = os.path.join(glacier_dir, "meta_data")
    meta_data_file_names = os.listdir(meta_data_dir)
    for meta_data_file_name in meta_data_file_names:
        with open(os.path.join(meta_data_dir,meta_data_file_name), mode = "r") as file:
            meta = file.read()
            split_meta = meta.split("\n")
            del split_meta[0]
            del split_meta[-1]
            for unformatted_meta in split_meta:
                truncated_meta = unformatted_meta.split(",\"")[1][:-1]
                meta_data = truncated_meta.replace("\'", "\"")
                meta_dict = json.loads(meta_data)
                meta_id_split = meta_dict['id'].split("_")
                
                landsat = "L" + str(int(meta_id_split[1][-2:]))
                date = meta_id_split[-1]
                year = date[:4]
                month = date[4:6]
                day = date[6:]
                glacier_pk = f"{glacier}_{year}-{month}-{day}_{landsat}_T1_TOA"
                
                cloud = meta_dict['properties']['cloud']
                cloud_cover = meta_dict['properties']['CLOUD_COVER']
                try: 
                    image_quality = meta_dict['properties']['IMAGE_QUALITY']
                except:
                    image_quality = ""
                
                try: 
                    image_quality_tirs = meta_dict['properties']['IMAGE_QUALITY_TIRS']
                except:
                    image_quality_tirs = ""
                    
                try: 
                    image_quality_oli = meta_dict['properties']['IMAGE_QUALITY_OLI']
                except:
                    image_quality_oli = ""
                       
                cloud_cover_land = meta_dict['properties']['CLOUD_COVER_LAND']
                system_index = meta_dict['properties']['system:index']
                utm_zone = meta_dict['properties']['UTM_ZONE']
                
                ee_meta_data = {"glacier_pk":glacier_pk, 
                               "cloud":cloud, 
                               "cloud_cover": cloud_cover,
                               "image_quality": image_quality,
                               "image_quality_tirs": image_quality_tirs,
                               "iamge_quality_oli": image_quality_oli,
                               "cloud_cover_land": cloud_cover_land,
                               "system_index": system_index,
                               "utm_zone": utm_zone
                              }
                ee_meta_data_df = ee_meta_data_df.append(ee_meta_data, ignore_index = True)                


In [30]:
ee_meta_data_df.to_csv(os.path.join(output_dir,"ee_metadata.csv"), index = False)